In [67]:
from lib import get_data, display_doc, print_question, display_dependencies

dat=get_data()
#print(dat)
print(print_question(dat,3,0))
display_doc(dat,3)

{'answers': [{'text': 'WEB SITE HOSTING AGREEMENT', 'answer_start': 225}], 'id': 'CENTRACKINTERNATIONALINC_10_29_1999-EX-10.3-WEB SITE HOSTING AGREEMENT__Document Name', 'question': 'Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract', 'is_impossible': False}


In [14]:
import stanza
import pandas as pd
from IPython.display import display, HTML

# Download the English models and initialize the pipeline
#stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,lemma,depparse')
#print(dat['data'][0]['paragraphs'][0]['context'])
txt=dat['data'][0]['paragraphs'][0]['context']
doc=nlp(txt)

2024-10-30 14:52:50 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-10-30 14:52:50 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2024-10-30 14:52:51 INFO: Using device: cuda
2024-10-30 14:52:51 INFO: Loading: tokenize
/home/maria/Documents/LegalRAG/.venv/lib/python3.12/site-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md

In [17]:
# Find dependents of the word "AGREEMENT"
dependents = []
for sentence in doc.sentences:
    for word in sentence.words:
        if word.text.upper() == "AGREEMENT":  # Match the word "AGREEMENT"
            for child in sentence.words:
                if child.head == word.id:  # Check if the current word is dependent on "AGREEMENT"
                    dependents.append({"Dependent": child.text, "Dependency Type": child.deprel})

# Display the results in a table
df = pd.DataFrame(dependents)
display(HTML(df.to_html(index=False)))


Dependent,Dependency Type
DISTRIBUTOR,compound
THIS,det
DISTRIBUTOR,compound
Agreement,appos
(,punct
the,det
"""",punct
"""",punct
),punct
into,case


In [20]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,lemma,depparse')

2024-10-30 15:02:07 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-10-30 15:02:07 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2024-10-30 15:02:07 INFO: Using device: cuda
2024-10-30 15:02:07 INFO: Loading: tokenize
2024-10-30 15:02:07 INFO: Loading: pos
2024-10-30 15:02:07 INFO: Loading: lemma
2024-10-30 15:02:07 INFO: Loading: depparse
2024-10-30 15:02:08 INFO: Done loading processors!


In [24]:
import stanza

def get_agreement_type(dat, doc_id):
    """
    Extract dependents of a target word with a specified dependency type from a Stanza parsed document.
    
    Parameters:
    - doc: Stanza parsed document
    - target_word (str): The word whose dependents you want to find
    - dep_type (str): The dependency type to filter by (default is "compound")
    
    Returns:
    - List of dependent words with the specified dependency type
    """
    #print(dat['data'][0]['paragraphs'][0]['context'])
    txt=dat['data'][doc_id]['paragraphs'][0]['context']
    doc=nlp(txt)
    dependents = []
    
    head="AGREEMENT"
    target_word=head
    dep_type="compound"

    for sentence in doc.sentences:
        for word in sentence.words:
            if word.text.upper() == target_word.upper():  # Match the target word (case-insensitive)
                for child in sentence.words:
                    if child.head == word.id and child.deprel == dep_type:  # Check dependency type
                        dependents.append(child.text)
    
    return dependents

# Usage example
# Assuming 'nlp' is the Stanza pipeline and 'txt' is the input text
compound_dependents = get_agreement_type(dat, 3)
print(compound_dependents)


['SITE', 'SITE']


In [ ]:
txt=dat['data'][0]['paragraphs'][0]['context']
doc=nlp(txt)

In [33]:
#Position in the document is important
def my_func(doc_ind):
    txt=dat['data'][doc_ind]['paragraphs'][0]['context']
    doc=nlp(txt)
    dependents = []
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.text.upper() == "AGREEMENT":  # Match the word "AGREEMENT"
                for child in sentence.words:
                    if child.head == word.id:  # Check if the current word is dependent on "AGREEMENT"
                        dependents.append({"Dependent": child.text, "Dependency Type": child.deprel})

    # Display the results in a table
    df = pd.DataFrame(dependents)
    display(HTML(df.to_html(index=False)))

my_func(3)

Dependent,Dependency Type
SITE,compound
HOSTING,amod
This,det
SITE,compound
HOSTING,amod
Agreement,appos
(,punct
"""",punct
this,det
"""",punct


In [49]:
import stanza

def get_agreement_type(dat, doc_id):
    """
    Extracts and returns the phrase formed by a target word (AGREEMENT or CONTRACT) and its 'compound' dependents,
    in the exact order they appear in the text, excluding any dependent word that is 'Exhibit' (case insensitive).
    
    Parameters:
    - dat (dict): The dataset containing text documents
    - doc_id (int): The document ID to access specific text data in 'dat'
    
    Returns:
    - List of phrases formed by the target word and its 'compound' dependents, in the order they appear
    """
    # Get the context text for the specified document ID
    txt = dat['data'][doc_id]['paragraphs'][0]['context']
    doc = nlp(txt)  # Process the text with Stanza
    results = []
    
    target_words = ["AGREEMENT", "CONTRACT"]
    dep_type = "compound"
    
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.text in target_words:  # Exact case match with "AGREEMENT" or "CONTRACT"
                phrase_parts = [(word.text, word.id)]
                
                # Collect compound dependents, excluding "Exhibit" (case insensitive)
                for child in sentence.words:
                    if child.head == word.id and child.deprel == dep_type and child.text.lower() != "exhibit":
                        phrase_parts.append((child.text, child.id))
                
                # Sort phrase parts by position to ensure they are in the original order
                phrase_parts = sorted(phrase_parts, key=lambda x: x[1])
                
                # Join the words in order and add to results
                ordered_phrase = " ".join([part[0] for part in phrase_parts])
                results.append(ordered_phrase)
    
    return results

# Usage example
# Assuming 'dat' is your dataset and 'nlp' is the initialized Stanza pipeline
phrases = get_agreement_type(dat, doc_id=1)
print(phrases)


['DISTRIBUTION AGREEMENT']


In [52]:
import stanza

def get_agreement_type(dat, doc_id):
    """
    Extracts and returns the phrase formed by a target word (AGREEMENT or CONTRACT) and all its recursive 'compound' dependents,
    in the exact order they appear in the text, excluding any dependent word that is 'Exhibit' (case insensitive).
    
    Parameters:
    - dat (dict): The dataset containing text documents
    - doc_id (int): The document ID to access specific text data in 'dat'
    
    Returns:
    - List of phrases formed by the target word and all its 'compound' dependents, in the order they appear
    """
    # Get the context text for the specified document ID
    txt = dat['data'][doc_id]['paragraphs'][0]['context']
    doc = nlp(txt)  # Process the text with Stanza
    results = []
    
    target_words = ["AGREEMENT", "CONTRACT"]
    dep_type = "compound"
    
    def collect_dependents(word, sentence):
        """
        Recursively collect all compound dependents of a word in the sentence.
        
        Parameters:
        - word: The word object to collect dependents for
        - sentence: The sentence containing the word and its dependents
        
        Returns:
        - List of tuples (text, id) of all dependents, including the word itself, in document order
        """
        dependents = [(word.text, word.id)]
        
        for child in sentence.words:
            if child.head == word.id and child.deprel == dep_type and child.text.lower() != "exhibit":
                dependents.extend(collect_dependents(child, sentence))
        
        return dependents

    # Find the target words and recursively collect their compound dependents
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.text in target_words:  # Exact case match with "AGREEMENT" or "CONTRACT"
                # Collect all dependents recursively and sort them by their original order in the text
                phrase_parts = collect_dependents(word, sentence)
                phrase_parts = sorted(phrase_parts, key=lambda x: x[1])  # Sort by position (id) in text
                
                # Join the words in order and add to results
                ordered_phrase = " ".join([part[0] for part in phrase_parts])
                results.append(ordered_phrase)
    
    return results

# Usage example
# Assuming 'dat' is your dataset and 'nlp' is the initialized Stanza pipeline
phrases = get_agreement_type(dat, doc_id=1)
print(phrases)


['DISTRIBUTION AGREEMENT']


In [50]:
import stanza

def analyze_dependencies(dat, doc_id, target_word=""):
    """
    Analyzes and prints dependency relations and heads for words in sentences containing the target word.
    
    Parameters:
    - dat (dict): The dataset containing text documents
    - doc_id (int): The document ID to access specific text data in 'dDISTRIBUTORat'
    - target_word (str): The target word to analyze dependencies for (default is "AGREEMENT")
    
    Prints:
    - The text, dependency relation, and head of each word in the relevant sentence
    """
    # Get the context text for the specified document ID
    txt = dat['data'][doc_id]['paragraphs'][0]['context']
    doc = nlp(txt)  # Process the text with Stanza

    for sentence in doc.sentences:
        for word in sentence.words:
            if word.text == target_word:  # Exact match with the target word
                print(f"\nAnalyzing dependencies in sentence with '{target_word}':\n")
                for w in sentence.words:
                    head_text = sentence.words[w.head - 1].text if w.head > 0 else "ROOT"
                    print(f"Word: {w.text}, DepRel: {w.deprel}, Head: {head_text} (ID: {w.head})")

# Usage example
# Assuming 'dat' is your dataset and 'nlp' is the initialized Stanza pipeline
analyze_dependencies(dat, doc_id=1, target_word="DISTRIBUTION")



Analyzing dependencies in sentence with 'DISTRIBUTION':

Word: PROMOTION, DepRel: root, Head: ROOT (ID: 0)
Word: AND, DepRel: cc, Head: AGREEMENT (ID: 4)
Word: DISTRIBUTION, DepRel: compound, Head: AGREEMENT (ID: 4)
Word: AGREEMENT, DepRel: conj, Head: PROMOTION (ID: 1)

Analyzing dependencies in sentence with 'DISTRIBUTION':

Word: 3., DepRel: dep, Head: DISTRIBUTION (ID: 2)
Word: DISTRIBUTION, DepRel: root, Head: ROOT (ID: 0)
Word: AND, DepRel: cc, Head: OBLIGATIONS (ID: 5)
Word: OTHER, DepRel: amod, Head: OBLIGATIONS (ID: 5)
Word: OBLIGATIONS, DepRel: conj, Head: DISTRIBUTION (ID: 2)


In [54]:
import stanza

def get_agreement_type(dat, doc_id):
    """
    Extracts and returns the phrase formed by a target word (AGREEMENT or CONTRACT) and all its recursive dependents,
    starting with those connected by the 'compound' relation, in the exact order they appear in the text.
    Excludes any word that is 'Exhibit' (case insensitive).
    
    Parameters:
    - dat (dict): The dataset containing text documents
    - doc_id (int): The document ID to access specific text data in 'dat'
    
    Returns:
    - List of phrases formed by the target word and all its dependents, in the order they appear
    """
    # Get the context text for the specified document ID
    txt = dat['data'][doc_id]['paragraphs'][0]['context']
    doc = nlp(txt)  # Process the text with Stanza
    results = []
    
    target_words = ["AGREEMENT", "CONTRACT"]
    dep_type = "compound"
    
    def collect_all_dependents(word, sentence):
        """
        Recursively collect all dependents of a word in the sentence, regardless of dependency type.
        
        Parameters:
        - word: The word object to collect dependents for
        - sentence: The sentence containing the word and its dependents
        
        Returns:
        - List of tuples (text, id) of all dependents, including the word itself, in document order
        """
        dependents = [(word.text, word.id)]
        
        for child in sentence.words:
            if child.head == word.id and child.text.lower() != "exhibit":
                dependents.extend(collect_all_dependents(child, sentence))
        
        return dependents

    # Find the target words and recursively collect their compound dependents' dependents
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.text in target_words:  # Exact case match with "AGREEMENT" or "CONTRACT"
                phrase_parts = [(word.text, word.id)]
                
                # Collect dependents that are connected by 'compound'
                for child in sentence.words:
                    if child.head == word.id and child.deprel == dep_type and child.text.lower() != "exhibit":
                        phrase_parts.extend(collect_all_dependents(child, sentence))
                
                # Sort phrase parts by position to ensure they are in the original order
                phrase_parts = sorted(phrase_parts, key=lambda x: x[1])
                
                # Join the words in order and add to results
                ordered_phrase = " ".join([part[0] for part in phrase_parts])
                results.append(ordered_phrase)
    
    return results

# Usage example
# Assuming 'dat' is your dataset and 'nlp' is the initialized Stanza pipeline
phrases = get_agreement_type(dat, doc_id=1)
print(phrases)


['DISTRIBUTION AGREEMENT']


In [68]:
# Define the text
text = """
1 Exhibit 10.3 I-on. (LOGO) www.i-on.com 561.394.9484 o 561.394-9773 fax 1733 avenida del sol, boca raton, florida, 33432 WEB SITE HOSTING AGREEMENT This WEB SITE HOSTING AGREEMENT 
"""

# Process the text with Stanza
doc = nlp(text)

# Prepare data for the DataFrame
data = []
for sentence in doc.sentences:
    for word in sentence.words:
        head_text = sentence.words[word.head - 1].text if word.head > 0 else "ROOT"
        data.append({
            "Word": word.text,
            "Head": head_text,
            "DepRel": word.deprel
        })

# Create DataFrame and display it as an HTML table
df = pd.DataFrame(data)
display(HTML(df.to_html(index=False)))

Word,Head,DepRel
1,Exhibit,nummod
Exhibit,ROOT,root
10.3,I,nummod
I,Exhibit,list
-,on,punct
on,I,nmod
.,Exhibit,punct
(,LOGO,punct
LOGO,I,appos
),LOGO,punct


In [69]:
import stanza

def get_agreement_type(dat, doc_id):
    """
    Extracts and returns the phrase formed by a target word (AGREEMENT or CONTRACT) and its relevant dependents,
    including all words that are part of the phrase based on 'compound', 'cc', and 'conj' relations,
    in the exact order they appear in the text, excluding any dependent word that is 'Exhibit' (case insensitive).

    Parameters:
    - dat (dict): The dataset containing text documents
    - doc_id (int): The document ID to access specific text data in 'dat'

    Returns:
    - List of phrases formed by the target word, its dependents, and connected words, in the order they appear
    """
    # Get the context text for the specified document ID
    txt = dat['data'][doc_id]['paragraphs'][0]['context']
    doc = nlp(txt)  # Process the text with Stanza
    results = []

    target_words = ["AGREEMENT", "CONTRACT"]

    for sentence in doc.sentences:
        for word in sentence.words:
            if word.text in target_words:  # Exact case match with "AGREEMENT" or "CONTRACT"
                phrase_parts = [(word.text, word.id)]

                # Collect dependents with 'compound' or 'cc' deprel, excluding "Exhibit" (case insensitive)
                for child in sentence.words:
                    if (child.head == word.id and 
                        child.deprel in ["compound", "cc"] and 
                        child.text.lower() != "exhibit"):
                        phrase_parts.append((child.text, child.id))

                # If the target word has a 'conj' deprel, include its head word and its dependents
                if word.deprel == "conj" and word.head > 0:
                    head_word = sentence.words[word.head - 1]
                    phrase_parts.append((head_word.text, head_word.id))

                    # Collect dependents of the head_word with deprels 'compound', 'cc', or 'amod', excluding 'Exhibit'
                    for child in sentence.words:
                        if (child.head == head_word.id and 
                            child.deprel in ["compound", "cc", "amod"] and 
                            child.text.lower() != "exhibit"):
                            phrase_parts.append((child.text, child.id))

                    # Collect any siblings of the target word connected via 'cc' or 'conj'
                    for sibling in sentence.words:
                        if (sibling.head == head_word.id and 
                            sibling.deprel in ["cc", "conj"] and 
                            sibling.id != word.id and 
                            sibling.text.lower() != "exhibit"):
                            phrase_parts.append((sibling.text, sibling.id))

                # Remove duplicates by converting to a set and back to a list
                phrase_parts = list(set(phrase_parts))

                # Sort phrase parts by position to ensure they are in the original order
                phrase_parts = sorted(phrase_parts, key=lambda x: x[1])

                # Join the words in order and add to results
                ordered_phrase = " ".join([part[0] for part in phrase_parts])
                results.append(ordered_phrase)

    return results

# Usage example
# Assuming 'dat' is your dataset and 'nlp' is the initialized Stanza pipeline
phrases = get_agreement_type(dat, doc_id=3)
print(phrases)


['SITE AGREEMENT', 'SITE AGREEMENT']


In [72]:
import stanza

def get_agreement_type(dat, doc_id):
    """
    Extracts and returns phrases formed by a target word (AGREEMENT or CONTRACT) and all its relevant dependents,
    including dependents of dependents, in the exact order they appear in the text.
    Excludes any dependent word that is 'Exhibit' (case insensitive).

    Parameters:
    - dat (dict): The dataset containing text documents.
    - doc_id (int): The document ID to access specific text data in 'dat'.

    Returns:
    - List of phrases formed by the target word, its dependents, and connected words, in the order they appear.
    """
    # Get the context text for the specified document ID
    txt = dat['data'][doc_id]['paragraphs'][0]['context']
    doc = nlp(txt)  # Process the text with Stanza
    results = []

    target_words = ["AGREEMENT", "CONTRACT"]
    # Include 'amod' in the list of dependency relations
    relevant_deprels = ["compound", "cc", "amod"]

    for sentence in doc.sentences:
        for word in sentence.words:
            if word.text == "Exhibit":  # Exclude 'Exhibit' as the target word
                continue
            if word.text in target_words:  # Exact case match with 'AGREEMENT' or 'CONTRACT'
                phrase_parts = []

                # Recursive function to collect dependents
                def collect_dependents_recursive(current_word):
                    collected = [(current_word.text, current_word.id)]
                    for child in sentence.words:
                        if (child.head == current_word.id and 
                            child.deprel in relevant_deprels and 
                            child.text.lower() != "exhibit"):
                            collected.extend(collect_dependents_recursive(child))
                    return collected

                # Start collecting from the target word
                phrase_parts.extend(collect_dependents_recursive(word))

                # If the target word has a 'conj' deprel, include its head and its dependents
                if word.deprel == "conj" and word.head > 0:
                    head_word = sentence.words[word.head - 1]
                    if head_word.text.lower() != "exhibit":
                        phrase_parts.extend(collect_dependents_recursive(head_word))

                # Remove duplicates by converting to a set and back to a list
                phrase_parts = list(set(phrase_parts))

                # Sort phrase parts by their IDs to maintain the original order
                phrase_parts = sorted(phrase_parts, key=lambda x: x[1])

                # Join the words in order and add to results
                ordered_phrase = " ".join([part[0] for part in phrase_parts])
                results.append(ordered_phrase)

    return results

# Usage example
# Assuming 'dat' is your dataset and 'nlp' is the initialized Stanza pipeline
phrases = get_agreement_type(dat, doc_id=1)
print(phrases)


['PROMOTION AND DISTRIBUTION AGREEMENT']
